# Delaunay Triangulation

In [ ]:
from triangulation.main import Triangulation

T = Triangulation(7, testing=True)
leaves = T.Delaunay()

In [ ]:
import plotly
print('plotly version: ', plotly.__version__, ' (must be > 3.6)')
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.io as pio
import plotly.figure_factory as ff
pio.templates.default = "plotly_white"
colors = px.colors.sequential.Agsunset
named_colorscales = px.colors.named_colorscales()
black_cs = [[0.0, 'black'], [1.0, 'black']]

In [ ]:
fig = go.Figure()

x = [p['lon'] for p in T.P]
y = [p['lat'] for p in T.P]

fig.add_trace(go.Scatter(x=x, y=y, mode='markers'))
    
fig.update_layout(
    xaxis=dict(title='X'),
    yaxis=dict(title='Y')
)

fig.show()

In [ ]:
fig = go.Figure()

x = [p['lon'] for p in T.P]
y = [p['lat'] for p in T.P]

for leaf in leaves:
    for i in [-1, 0, 1]:
        x = [T.P[leaf['vtx_ids'][i]]['lon'], T.P[leaf['vtx_ids'][i+1]]['lon']]
        y = [T.P[leaf['vtx_ids'][i]]['lat'], T.P[leaf['vtx_ids'][i+1]]['lat']]
        fig.add_trace(go.Scatter(x=x, y=y, mode='lines', line=dict(color='black')))
    
fig.update_layout(
    xaxis=dict(title='X'),
    yaxis=dict(title='Y')
)

fig.show()